<a href="https://colab.research.google.com/github/Jun-629/20MA573/blob/master/src/Hw10_MRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider 2-d PDE
$$\frac{1}{2} \Delta v(x) - v(x) + x_{1}^2 + x_{2}^2 - x_{1} - x_{2} - \frac{3}{2} = 0, x \in O=(0,1)^2$$
with its boundary data
$$v(x) = (x_{1} - \frac{1}{2})^2 + (x_{2} - \frac{1}{2})^2, x \notin O.$$
The exact solution is 
$$v(x) = (x_{1} - \frac{1}{2})^2 + (x_{2} - \frac{1}{2})^2.$$

- Idendify $\mathrm{MRP}$ with $\mathrm{CFD}$ in the form of 

\begin{align}
v(x) = \gamma \{ \ell ^h (x) + \sum_{i=1}^{d} p^h (x + he_{i}|x)v(x + he_{i}) + p^h(x-he_{i}|x)v(x - he_{i}) \} \tag{1} \\
\end{align}

__Soln:__

First, we change the form of the equation (1):

\begin{align}
v(x) = \gamma \cdot \ell ^h (x) + \gamma \cdot \sum_{i=1}^{2} p^h (x + he_{i}|x)v(x + he_{i}) + p^h(x-he_{i}|x)v(x - he_{i}) \tag{2} \\
\end{align}

Also, we already calculate that
$$\gamma = \frac{2}{2+h^{2}}, \quad \ell^{h}(x) = \frac{h^{2}}{2} [(x_{1} - \frac{1}{2})^{2} + (x_{2} - \frac{1}{2})^{2} -2], \quad p^{h}(x\pm he_{i}|x) = \frac{1}{4}.$$
Then compare (2) to the Bellman Equation:
$$v(s) = R(s) + \gamma \sum_{s' \in \mathcal S} P(s,s')v(s'),$$
we can deduce that

$$P(s,s') =
\begin{cases} 
\frac{1}{4},  & \mbox{if } ||s'-s||_1 = h \\
0, & \mbox{otherwise}
\end{cases}$$

and
$$ R(s) = \gamma \cdot \ell^h(s) = \frac{h^2}{2+h^2} [(x_{1} - \frac{1}{2})^{2} + (x_{2} - \frac{1}{2})^{2} -2] $$

- For $h$ = 1/8, compute $\mathrm{CFD}$ solution by value iteration.
- For $h$ = 1/8, compute $\mathrm{CFD}$ solution by Monte-Carlo method.
- For $h$ = 1/8, compute $\mathrm{CFD}$ solution by $\mathrm{TD}$ method.
- Compare above three methodsand conclude your observations.


In [96]:
# Value Iteration

import numpy as np

def gamma(N):
  h = 1/N
  gamma_value = 2/(2 + h**2)
  return gamma_value
def l(N):
  h = 1/N
  l_h = np.zeros([N+1, N+1])
  for i in range(N+1):
    for j in range(N+1):
      if i == 0 or i == N or j == 0 or j == N:
        l_h[i, j] = 0
      else:
        l_h[i, j] = h**2 / 2 *  ((i/N - 0.5)**2 + (j/N - 0.5)**2 - 2)
  return l_h
def F(N, u):
  h = 1/N
  l_h = l(N)
  v = np.zeros([N+1, N+1])
  for i in range(0, N+1):
    for j in range(0, N+1):
      if i == 0 or i == N or j == 0 or j == N:
        v[i, j] = u[i, j]
      else:
        v[i, j] = gamma(N) * l_h[i, j] + gamma(N) * (u[i+1, j] + u[i, j+1] + u[i, j-1] + u[i-1, j]) / 4
  return v
def initial_value(N):
  u = np.zeros([N+1, N+1])
  for i in range(0, N+1):
    for j in range(0, N+1):
      if i == 0 or i == N or j == 0 or j == N:
        u[i, j] = ((i/N - 0.5)**2 + (j/N - 0.5)**2)
      else:
        u[i, j] = 0
  return u
def value_iteration(N, error_hat = 0.0001):
  v = initial_value(N)
  error = 1
  step = 0
  while error > error_hat:
    step +=1
    u = v
    v = F(N, u)
    error = np.max(np.abs(u - v))
  return [error, step, v]
VI_soln = value_iteration(8)[2]
# print(VI_soln)

[[ 0.5         0.390625    0.3125      0.265625    0.25        0.265625
   0.3125      0.390625    0.5       ]
 [ 0.390625    0.28111535  0.20288037  0.15592491  0.14027904  0.15592491
   0.20288037  0.28111535  0.390625  ]
 [ 0.3125      0.20288037  0.12454026  0.0775344   0.06184983  0.0775344
   0.12454026  0.20288037  0.3125    ]
 [ 0.265625    0.15592491  0.0775344   0.03046517  0.01478977  0.03046517
   0.0775344   0.15592491  0.265625  ]
 [ 0.25        0.14027904  0.06184983  0.01478977 -0.00091948  0.01478977
   0.06184983  0.14027904  0.25      ]
 [ 0.265625    0.15592491  0.0775344   0.03046517  0.01478977  0.03046517
   0.0775344   0.15592491  0.265625  ]
 [ 0.3125      0.20288037  0.12454026  0.0775344   0.06184983  0.0775344
   0.12454026  0.20288037  0.3125    ]
 [ 0.390625    0.28111535  0.20288037  0.15592491  0.14027904  0.15592491
   0.20288037  0.28111535  0.390625  ]
 [ 0.5         0.390625    0.3125      0.265625    0.25        0.265625
   0.3125      0.390625    0

In [101]:
# Monte-Carlo Method
import numpy as np
import random as rd

N = 8
h = 1/N
alpha = 0.04 

s = [x for x in range(1,N)]
#rd.seed(1)
s0 = [rd.choice(s), rd.choice(s)]

S_list = [s0]
print(S_list)         # Still change S_list[0] even though set it firstly
ei = [-1,1]
T = 0                 # T is the stopping time

while s0[0]>0 and s0[0]<N and s0[1]>0 and s0[1]<N:
  s0[0] += rd.choice(ei)
  s0[1] += rd.choice(ei)
  S_list.append([s0[0],s0[1]])
  T += 1              # S_list is the state space excepet S_list[0]
print(S_list)

R = np.zeros(T+1)
v = 0
for i in range(T+1):
  x1 = S_list[i][0]
  x2 = S_list[i][1]
  R[i] = h**2/(2+h**2)*((x1/N-0.5)**2+(x2/N-0.5)**2-2)        # The reward list, (terminal is equal to 0?)
G = R[T]
V_list = [0]
for t in range(T-1,0,-1):
  G = R[t] + gamma(N) * G
  v += alpha * (G - v)
  V_list.append(v)               #Owing to the bug in S_list[0], there is no value when T=1
print(V_list)

#print(l(4)*gamma(4))   # This is the complete R(s)


[[3, 5]]
[[8, 4], [2, 6], [3, 7], [4, 6], [5, 5], [6, 4], [7, 3], [8, 4]]
[0, -0.0011101346072952348, -0.0027680333294669143, -0.004956885063166739, -0.007641131773511759, -0.010767368909141649, -0.014323348168597734]
